In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_df = pd.read_excel('美債20年.xlsx')
data_df.rename(columns={"淨值":"國泰淨值", "折溢價":"國泰折溢價", "折溢價%":"國泰折溢價%"},inplace=True)
data_df.rename(columns={"淨值.1":"元大淨值", "折溢價.1":"元大折溢價", "折溢價%.1":"元大折溢價%"},inplace=True)
data_df = data_df.drop(columns=['Unnamed: 11', 'Unnamed: 13', 'Unnamed: 5'])

data_df['淨值比'] = data_df['元大淨值'] / data_df['國泰淨值'] # 一單位元大可以買多少單位國泰
data_df['cash_flow'] = data_df['元大市價'] - (data_df['國泰市價'] * data_df['淨值比'])  # 當天借券(元大)賣出一單位，買入等比例的國泰
# 假如該天為正，代表有套利獲利，只要下一天或之後的現金流比該天小，就能買回元大、賣出國泰，完成套利並結算。
data_df.head()

,日期,國泰市價,國泰淨值,國泰折溢價,國泰折溢價%,元大市價,元大淨值,元大折溢價,元大折溢價%,元大減國泰折溢價百分比,cost,淨值比,cash_flow
0,2023-05-30,32.44,32.2837,0.16,0.50,31.82,30.8254,0.99,3.21,2.71,-0.62,0.954829,0.845360
1,2023-05-29,32.26,31.9828,0.28,0.88,31.81,30.6444,1.17,3.82,2.94,-0.45,0.958153,0.900000
2,2023-05-26,32.18,32.0412,0.14,0.44,31.77,30.5907,1.18,3.86,3.42,-0.41,0.954730,1.046783
3,2023-05-25,32.17,32.0480,0.12,0.37,31.63,30.5951,1.03,3.37,3.00,-0.54,0.954665,0.918431
4,2023-05-24,32.34,32.3388,0.00,0.00,31.76,30.8502,0.91,2.95,2.95,-0.58,0.953969,0.908655


In [ ]:
transcation_cost_rate = 0.001425
lending_cost_rate = 0.00008
# 手續費 0.1425%，融券費是0.008%
t = 0
for i in range(len(data_df)-1, -1, -1):
  for j in range(i, -1, -1):
    profit = data_df['cash_flow'].iloc[i] - data_df['cash_flow'].iloc[j]
    # 第i天的交易成本
    cost_i = data_df['元大市價'].iloc[i] * lending_cost_rate + data_df['國泰市價'].iloc[i] * data_df['淨值比'].iloc[i] * transcation_cost_rate
    # 第j天的交易成本
    cost_j = data_df['元大市價'].iloc[j] * lending_cost_rate + data_df['國泰市價'].iloc[j] * data_df['淨值比'].iloc[j] * transcation_cost_rate
    profit -= (cost_i + cost_j)
    if profit > 0:
      date_i = data_df['日期'].iloc[i].strftime('%Y/%m/%d')
      date_j = data_df['日期'].iloc[j].strftime('%Y/%m/%d')
      print(f'在 {date_i} 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 {date_j} 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 {profit} 的無風險利潤')
      print(' ')
      t += 1
      # break
print('總套利機會: ', t)

在 2023/04/18 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/04/19 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.036776280940265804 的無風險利潤
 
在 2023/04/20 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/04/26 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.005332066983269004 的無風險利潤
 
在 2023/04/21 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/04/26 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.016946252580841675 的無風險利潤
 
在 2023/04/24 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/04/26 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.05200565968993967 的無風險利潤
 
在 2023/05/10 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/05/12 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.11125383941022934 的無風險利潤
 
在 2023/05/11 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/05/12 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.0934698755821945 的無風險利潤
 
在 2023/05/18 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/05/29 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.0006600310327424808 的無風險利潤
 
在 2023/05/18 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ETF, 在 2023/05/30 賣出 國泰美債ETF, 買回 元大美債ETF 還券, 可得到 0.055206954157843866 的無風險利潤
 
在 2023/05/19 借券賣出 元大美債ETF, 並買入等比例單位的 國泰美債ET